In [9]:
import inspect
import json
from typing import Any, Callable, Dict, List, Optional, Type
from dslmodel import DSLModel



def get_parameters_schema(func: Callable) -> Dict[str, Any]:
    sig = inspect.signature(func)
    properties = {}
    required = []
    for name, param in sig.parameters.items():
        annotation = param.annotation
        if annotation is inspect.Parameter.empty:
            param_type = 'string'
        else:
            param_type = get_json_schema_type(annotation)
        properties[name] = {
            'type': param_type,
            'description': '',  # Optionally, add parameter descriptions
        }
        if param.default is inspect.Parameter.empty:
            required.append(name)
    schema = {
        'type': 'object',
        'properties': properties,
        'required': required,
        'additionalProperties': False,
    }
    return schema

def get_json_schema_type(py_type: Type) -> str:
    # Map Python types to JSON Schema types
    if py_type == str:
        return 'string'
    elif py_type == int:
        return 'integer'
    elif py_type == float:
        return 'number'
    elif py_type == bool:
        return 'boolean'
    elif py_type == dict:
        return 'object'
    elif py_type == list:
        return 'array'
    else:
        return 'string'  # Default to string if type is unknown

In [8]:


class ToolMixin:
    def __init__(self):
        self._tools: List[Callable] = []
        self.collect_tools()

    def collect_tools(self):
        # Collect methods decorated with @tool
        self._tools = []
        for name, method in inspect.getmembers(self, predicate=inspect.ismethod):
            if hasattr(method, '_tool_data'):
                self._tools.append(method)

    def get_tool_definitions(self) -> List[Dict[str, Any]]:
        # Generate tool definitions for the OpenAI API
        tool_definitions = []
        for method in self._tools:
            tool_data = method._tool_data
            tool_definitions.append({
                'type': 'function',
                'function': {
                    'name': tool_data['name'],
                    'description': tool_data['description'],
                    'parameters': tool_data['parameters'],
                }
            })
        return tool_definitions

    def get_tool_method(self, function_name: str) -> Optional[Callable]:
        # Find the method corresponding to the function name
        for method in self._tools:
            if method._tool_data['name'] == function_name:
                return method
        return None

    def process_response(self, response: Dict[str, Any]) -> List[Dict[str, Any]]:
        # Process the response from the OpenAI API
        results = []
        choices = response.get('choices', [])
        for choice in choices:
            message = choice.get('message', {})
            tool_calls = message.get('tool_calls', [])
            for tool_call in tool_calls:
                function_name = tool_call['function']['name']
                arguments_str = tool_call['function']['arguments']
                try:
                    arguments = json.loads(arguments_str)
                except json.JSONDecodeError:
                    # Handle JSON parsing error
                    continue
                method = self.get_tool_method(function_name)
                if method:
                    try:
                        result = method(**arguments)
                        # Prepare the message to send the result back to the model
                        result_message = {
                            "role": "tool",
                            "content": json.dumps(result),
                            "tool_call_id": tool_call['id']
                        }
                        results.append(result_message)
                    except Exception as e:
                        # Handle execution errors
                        error_message = {
                            "role": "tool",
                            "content": json.dumps({"error": str(e)}),
                            "tool_call_id": tool_call['id']
                        }
                        results.append(error_message)
                else:
                    # Handle unknown function
                    error_message = {
                        "role": "tool",
                        "content": json.dumps({"error": f"Function {function_name} not found."}),
                        "tool_call_id": tool_call['id']
                    }
                    results.append(error_message)
        return results



In [3]:
import httpx
from typing import List, Dict, Any, Optional

from dslmodel import DSLModel, init_instant, init_text


class Cat(DSLModel):
    id: str
    name: Optional[str] = None
    url: str
    width: int
    height: int
    
class CatBreed(DSLModel):
    name: str
    description: str

# Define the base URL for The Cat API
CAT_API_URL = "https://api.thecatapi.com/v1"

# Tool to fetch a random cat image
@tool(name="fetch_random_cat_image", description="Fetch a random cat image")
def fetch_random_cat_image() -> Cat:
    with httpx.Client() as client:
        response = client.get(f"{CAT_API_URL}/images/search")
        return Cat(**response.json()[0])

# Tool to fetch cat breeds information
@tool(name="fetch_cat_breeds", description="Fetch information about cat breeds")
def fetch_cat_breeds(limit: Optional[int] = 5) -> List[Dict[str, Any]]:
    with httpx.Client() as client:
        response = client.get(f"{CAT_API_URL}/breeds")
        if response.status_code == 200:
            breeds = response.json()[:limit]  # Limit the number of breeds
            breed_info = [CatBreed(name=breed["name"], description=breed["description"]) for breed in breeds]
            return breed_info
        else:
            raise Exception(f"Failed to fetch cat breeds. Status code: {response.status_code}")

In [4]:
init_instant()

cat = fetch_random_cat_image()
print(cat)
cat2 = Cat.from_prompt(f"{cat} change name to Cali. Change the output")
print(cat2)

id='DbwiefiaY' name=None url='https://cdn2.thecatapi.com/images/DbwiefiaY.png' width=1200 height=627
id='DbwiefiaY' name='Cali' url='https://cdn2.thecatapi.com/images/DbwiefiaY.png' width=1200 height=627


In [5]:
fetch_cat_breeds()

[CatBreed(name='Abyssinian', description='The Abyssinian is easy to care for, and a joy to have in your home. They’re affectionate cats and love both people and other animals.'),
 CatBreed(name='Aegean', description='Native to the Greek islands known as the Cyclades in the Aegean Sea, these are natural cats, meaning they developed without humans getting involved in their breeding. As a breed, Aegean Cats are rare, although they are numerous on their home islands. They are generally friendly toward people and can be excellent cats for families with children.'),
 CatBreed(name='American Bobtail', description='American Bobtails are loving and incredibly intelligent cats possessing a distinctive wild appearance. They are extremely interactive cats that bond with their human family with great devotion.'),
 CatBreed(name='American Curl', description='Distinguished by truly unique ears that curl back in a graceful arc, offering an alert, perky, happily surprised expression, they cause people 

In [7]:
class CatTools(ToolMixin):
    # Tool to fetch a random cat image
    @tool(name="fetch_random_cat_image", description="Fetch a random cat image")
    def fetch_random_cat_image(self) -> Cat:
        with httpx.Client() as client:
            response = client.get(f"{CAT_API_URL}/images/search")
            return Cat(**response.json()[0])

    # Tool to fetch cat breeds information
    @tool(name="fetch_cat_breeds", description="Fetch information about cat breeds")
    def fetch_cat_breeds(self, limit: Optional[int] = 5) -> List[Dict[str, Any]]:
        with httpx.Client() as client:
            response = client.get(f"{CAT_API_URL}/breeds")
            if response.status_code == 200:
                breeds = response.json()[:limit]  # Limit the number of breeds
                breed_info = [CatBreed(name=breed["name"], description=breed["description"]) for breed in breeds]
                return breed_info
            else:
                raise Exception(f"Failed to fetch cat breeds. Status code: {response.status_code}")
            

tools = CatTools()
tools.get_tool_definitions()

[{'type': 'function',
  'function': {'name': 'fetch_cat_breeds',
   'description': 'Fetch information about cat breeds',
   'parameters': {'type': 'object',
    'properties': {'self': {'type': 'string', 'description': ''},
     'limit': {'type': 'string', 'description': ''}},
    'required': ['self'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'fetch_random_cat_image',
   'description': 'Fetch a random cat image',
   'parameters': {'type': 'object',
    'properties': {'self': {'type': 'string', 'description': ''}},
    'required': ['self'],
    'additionalProperties': False}}}]

In [24]:
from pydantic import Field

class Tool(DSLModel):
    name: str
    description: Optional[str] = None
    method: Callable = None
    
    
class ChosenTool(DSLModel):
    """Name of the def to call"""
    name: str = Field(..., description="Name of the function to call")
    kwargs: dict = Field(default={}, description="Keyword arguments to pass to the function")


class CatTools:
    current_cat: Cat = Field(default=None)
    current_breeds: list[CatBreed] = Field(default=[])

    
    
    def fetch_random_cat_image(self, **kwargs) -> Cat:
        with httpx.Client() as client:
            response = client.get(f"{CAT_API_URL}/images/search", **kwargs)
            return Cat(**response.json()[0])
    
    def fetch_cat_breeds(self, limit: Optional[int] = 5, **kwargs) -> List[Dict[str, Any]]:
        with httpx.Client() as client:
            response = client.get(f"{CAT_API_URL}/breeds", **kwargs )
            breeds = response.json()[:limit]  # Limit the number of breeds
            breed_info = [CatBreed(name=breed["name"], description=breed["description"]) for breed in breeds]
            return breed_info
       
    def from_call(self, prompt: str) -> 'CatTools': 
        chose = ChosenTool.from_prompt(f"{self}\n{prompt}")
        return chose
            

tools = CatTools().from_call("Get me 10 cat breeds")
print(tools)

KeyboardInterrupt: 